In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from yellowbrick.regressor import ResidualsPlot
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [18]:
class run_regression:

  def load_dataset(self):
    df = pd.read_excel("regression_all_day.xlsx")
    df = df[df['cluster_id'] != -99]
    df = df[df['mean_avail_all'] > 1]
    df = df[df['all_pickups'] > 1]

    df['ln_avail'] = np.log(df['mean_avail_all'].values)
    df['ln_pickups'] = np.log(df['all_pickups'].values)
    df = df.drop(columns = ['mean_avail_all', 'all_pickups', 'street'])
    df = df.reset_index(drop = True)
  
    return df

  def change_poi_col_names(self, df):
    col_poi = df.drop(columns = ['ln_pickups', 'ln_avail', 'cluster_id']).columns
    dict1 = {}

    count = 0
    for col in col_poi:
      dict1[col] = 'poi_'+str(count)
      count += 1
    
    df = df.rename(columns = dict1)
    return df

  def get_dummies1(self, df):
    unique_cluster = df['cluster_id'].unique()
    df = pd.get_dummies(columns = ['cluster_id'], data = df)

    col_cluster_id_dummies = []
    for i in range(len(unique_cluster)):
      col_cluster_id_dummies.append('cluster_id_' + str(unique_cluster[i]))
    
    return df, col_cluster_id_dummies
  
  def transform(self, df, col_name1, dummy_col_name_array):
    dict1 = {}
    for x in dummy_col_name_array:
      temp_df = df[df[x] == 1]
      dict1[x] = np.mean(temp_df[col_name1].values)

    dict_keys = list(dict1.keys())
    X = df[dict_keys].values
    T = np.asarray(list(dict1.values()))
    T = T.reshape(len(T), 1)
        
    transformed_values = np.matmul(X, T)
        
    return transformed_values

  def func_col_to_be_transformed(self):
    col_to_be_transformed = ['ln_avail', 'ln_pickups']
    for i in range(31):
      temp_col_name = 'poi_' + str(i)
      col_to_be_transformed.append(temp_col_name)
        
    return col_to_be_transformed

  def func_transformed_column_names(self, col_to_be_transformed, dummy_col_name_array, df):
    transformed_column_names = []
        
    for col_name1 in col_to_be_transformed:
      temp_col_name1 = 'diff_' + col_name1

      transformed_values = self.transform(df, col_name1, dummy_col_name_array)

      df[temp_col_name1] = transformed_values
      transformed_column_names.append(temp_col_name1)
        
    return df, transformed_column_names
  
  def modify_dataset(self, df, transformed_column_names):
    df1 = df[transformed_column_names]
    return df1


  def regression1(self, df1):
    col_remain = df1.drop(columns = ['diff_ln_avail', 'diff_ln_pickups']).columns
    eq_string = col_remain[0]
        
    for i in range(1, len(col_remain)):
      eq_string = eq_string + "+" + col_remain[i]
            
    formula = 'diff_ln_pickups ~ diff_ln_avail' + '+' + eq_string
    mod = smf.ols(formula= formula, data=df1)
    res = mod.fit()
    results_summary = res.summary()

    return results_summary

  def drop_func_p_value(self, results_summary, df1):
    results_as_html = results_summary.tables[1].as_html()
        
    df_results1 = pd.read_html(results_as_html, 
                                   header = 0, index_col = 0)[0]
    df_results1 = df_results1.reset_index()
        
    dict_p = dict(zip(df_results1['index'].values, 
                          df_results1['P>|t|'].values))
        
    col_drop1 = []
    for key, value in dict_p.items():
      if value > 0.005:
        col_drop1.append(key)
        
    df2 = df1.drop(columns = col_drop1)
    return df2


  def main(self):
    df = self.load_dataset()
    df = self.change_poi_col_names(df)
    df, dummy_col_name_array = self.get_dummies1(df)
    col_to_be_transformed = self.func_col_to_be_transformed()
    df, transformed_column_names = self.func_transformed_column_names(col_to_be_transformed, 
                                                                      dummy_col_name_array, df)
    
    df1 = self.modify_dataset(df, transformed_column_names)
    results_summary = self.regression1(df1)

    df2 = self.drop_func_p_value(results_summary, df1)
    results_summary2 = self.regression1(df2)
        
    df3 = self.drop_func_p_value(results_summary2, df2)
    results_summary3 = self.regression1(df3)

    print(results_summary3)
    

    


In [19]:
temp_class = run_regression()
temp_class.main()


                            OLS Regression Results                            
Dep. Variable:        diff_ln_pickups   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     174.8
Date:                Mon, 29 Mar 2021   Prob (F-statistic):          1.38e-137
Time:                        02:17:24   Log-Likelihood:                -89.699
No. Observations:                 580   AIC:                             195.4
Df Residuals:                     572   BIC:                             230.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         4.1928      0.046     91.280